In [21]:
#Figure for the manifold plot

#The idea is to save the final 4100 state trajectory from the algorithm, 
#and at each point simulate forward in time without control to ensure that 
#at every timestep (if control were to be lost), we would escape to the right. 
#Do this for both the Earth Moon system and the Saturn Enceladus system

In [22]:
using LinearAlgebra
using DifferentialEquations
using Plots
using DelimitedFiles
using ForwardDiff

In [23]:
#parameters for Earth Moon System
μ = 1.901109735892602e-7
μ1 = 1-μ
μ2 = μ

#position of mass 1
pose_m1 = [-μ, 0, 0]

#position of mass 2
pose_m2 = [1-μ, 0, 0]

#value obtained from 3BP textbook (original)
L = 238529 #in km - distance between centers of m1 and m2
#V_ = 1.025 #in km/s - orbital velocity of m1
#T_ = 2.361e6 #in seconds - orbital period of m1 and m2


#working
#########################################################################################
#updated for time unit to be in days
T_ = 18913/86400 #in days - orbital period of m1 and m2
time_scale = T_ #/(2*pi)
#no need to divide by 2 pi for this time scale!

#time unit of days
V_ = L/time_scale
acceleration_scale = L/((time_scale)^2)
##########################################################################################

#state of the L1 and L2 Lagrange points (computed offline)
XL1 = [0.99601828, 0, 0, 0, 0, 0]
XL2 = [1.00399194, 0, 0, 0, 0, 0]

6-element Vector{Float64}:
 1.00399194
 0.0
 0.0
 0.0
 0.0
 0.0

In [24]:
function effective_potential(X)
    x = X[1]
    y = X[2]
    z = X[3]

    r1 = sqrt((x+μ2)^2 + y^2 + z^2) 
    r2 = sqrt((x-μ1)^2 + y^2 + z^2)
    #assuming m3 is unit mass
   
    U = (-μ1/r1)-(μ2/r2)-0.5*(x^2+y^2)
   
    return U
    
end

effective_potential (generic function with 1 method)

In [25]:
function three_body_prob_dynamics(x)
    
    q = zeros(eltype(x),3)
    v = zeros(eltype(x),3)
    q = x[1:3]
    v = x[4:6]

    ẋ = zeros(eltype(x),6)
    U_q = zeros(eltype(x),3)
    
    U_q = (ForwardDiff.gradient(_x -> effective_potential(_x), q))
        
    ẋ[1:3] = v
    ẋ[4] = 2*v[2] - U_q[1] 
    ẋ[5] = -2*v[1] - U_q[2]
    ẋ[6] = -U_q[3]

    return ẋ
end

three_body_prob_dynamics (generic function with 1 method)

In [26]:
#inputs are in the custom SI unit scale

function three_body_prob_dynamics_scaled(x)

    q_original = zeros(eltype(x),3)
    v_original = zeros(eltype(x),3)
    
    q_original = x[1:3]/L 
    v_original = x[4:6]/V_
    
    x_original = [q_original; v_original]

    #original is in the CR3BP units
    ẋ_original = zeros(eltype(x),6)

    #calculate the original xdot (no scaling)

    ẋ_original = three_body_prob_dynamics(x_original)

    #then scale the output
    v_scaled = ẋ_original[1:3]*V_
    
    a_scaled = ẋ_original[4:6]*acceleration_scale

    ẋ_scaled = [v_scaled; a_scaled]

    return ẋ_scaled

end

three_body_prob_dynamics_scaled (generic function with 1 method)

In [27]:
pose_m2*L 

3-element Vector{Float64}:
 238528.95465301958
      0.0
      0.0

In [28]:
#solve using DifferentialEquations.jl
function justdynamics_DFJL!(du, u, p, t)
    
    du[1:6] = three_body_prob_dynamics_scaled(u[1:6])
    
end

justdynamics_DFJL! (generic function with 1 method)

In [29]:
#gets the state and for the entire solution

function get_state(solution)
    
    N = size(solution.u)[1]

    all_states = zeros(6, N)

    for i=1:N
        all_states[:,i] = solution.u[i][1:6]
    end
    
    #all states and all stm are functions of t
    #solution.t is the time
    return all_states
end

get_state (generic function with 1 method)

In [30]:
#integrate just the dynamics
function just_dynamics_integrate(x_0, period)

    condition(u,t,integrator) = u[1] - 240000
    
    affect!(integrator) = terminate!(integrator)
     
    cb = ContinuousCallback(condition, affect!)
    
    tspan = (0.0, period)
    prob = ODEProblem(justdynamics_DFJL!, x_0, tspan)
    sol = solve(prob, TsitPap8(), abstol=1e-12, reltol=1e-12, callback=cb)
    
    return sol
    
end

just_dynamics_integrate (generic function with 1 method)

In [31]:
#whole x trajectory
xtraj  = readdlm("xtraj_enceledus_ctg.txt", ',')
#xtraj  = readdlm("xtraj_enceledus_tube.txt", ',')

6×4100 Matrix{Float64}:
     2.39588e5        2.39585e5  …      2.39585e5      2.39588e5
     2.88578e-15    -70.7208           70.712         -0.000555346
   226.168          224.152           223.96         225.987
    -9.46749e-10   -292.525           291.143          0.00566267
 -4203.74         -4159.35          -4157.2        -4204.95
    -9.76991e-10   -238.563      …    239.803         -0.0182633

In [32]:
x_reference  = readdlm("enceladus_reference.txt", ',')

6×41 Matrix{Float64}:
     2.39588e5        2.39585e5  …      2.39585e5      2.39588e5
     2.88578e-15    -70.7106           70.7106         4.0834e-8
   226.168          224.14            224.14         226.168
    -9.46749e-10   -291.102           291.102         -1.25936e-6
 -4204.83         -4157.08          -4157.08       -4204.83
    -9.76991e-10   -240.012      …    240.012         -2.11349e-7

In [33]:
escape_traj = Matrix{Float64}[]
dims_escape = zeros(2, 4100)

2×4100 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [34]:
days_time = 0.6752182741104091*2.15

1.4517192893373796

In [35]:
#at each of these points in x traj, simulate forward by a bit and hopefully all escape to the right ..

for i=1:4100

    integrate_sol = just_dynamics_integrate(xtraj[:,i], days_time)

    x_escape = get_state(integrate_sol)

    dims_escape[1,i] = size(x_escape)[1]

    dims_escape[2,i] = size(x_escape)[2]

    push!(escape_traj, x_escape)

end

In [41]:
plot(x_reference[1,:], x_reference[2,:], label=nothing)

for i=2:4100
    plot!(escape_traj[i][1,:], escape_traj[i][2,:], label=nothing, color=:red, fontfamily="Computer Modern", alpha=0.25) 
end

plot!(escape_traj[1][1,:], escape_traj[1][2,:], label="Exit Trajectories", color=:red, fontfamily="Computer Modern", alpha=0.8) 

plot!(x_reference[1,:], x_reference[2,:], label="Reference Trajectory",
 linewidth=5, color=:blue, fontfamily="Computer Modern", xguidefontsize=10, xtickfontsize=10, 
 yguidefontsize=10, ytickfontsize=10)

 title!("Exit Trajectories")
 xlabel!("X (km)")
 ylabel!("Y (km)")

 exit_trajectories = scatter!([pose_m2[1]*L], [0], label = "Enceladus", color=:orange, fontfamily="Computer Modern", grid=false) 


#savefig("exit_trajectories_enceladus.pdf") 

"/home/faustovega/Research/convex_mpc_stationkeeping/exit_trajectories_enceladus.pdf"

In [37]:
#savefig(exit_trajectories, "exit_trajectories_saturn_enceladus_tube")

In [38]:
pose_m2[1]*L

238528.95465301958

In [39]:
left = zeros(4099)

for i=2:4100

    left[i-1] = any(x -> x < 238528.95465301958, escape_traj[i][1,:]) 

end

In [40]:
1 - sum(left)/4099 

0.8287387167601854